# Traffic Sign Classification training

In [ ]:
# !pip install opencv-contrib-python
# !pip install numpy
# !pip install scikit-learn
#!pip install scikit-image==0.17.2 
# !pip install imutils
# !pip install matplotlib
# !pip install tensorflow

In [ ]:
!pip install matplotlib

In [1]:
base_path = "../Dataset/GTSRB_CNN"
save_path = "./trafficsignnet.model"
plot_path = "./plot.png"
test_path = "./Test/"
example_path = "./examples"

In [2]:
# import the necessary packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
class TrafficSignNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        # CONV => RELU => BN => POOL
        model.add(Conv2D(8, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # first set of (CONV => RELU => CONV => RELU) * 2 => POOL
        model.add(Conv2D(16, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(16, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
                  
        # second set of (CONV => RELU => CONV => RELU) * 2 => POOL
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
                  
        # first set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
                  
        # second set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
                  
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
                  
        return model

In [3]:
import matplotlib
matplotlib.use("Agg")
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
from skimage import transform
from skimage import exposure
from skimage import io
import matplotlib.pyplot as plt
import argparse
import random
import os
import time
import cv2

In [4]:
#function to load the dataset from the csv.
def load(basePath, csvPath):
    
    data = []
    labels = []
    rows = open(csvPath).read().strip().split("\n")[1:]
    #shuffling rows to shuffle classes
    random.shuffle(rows)
    
    # loop over the rows
    for (i, row) in enumerate(rows):
        if i % 1000 == 0:
            print("[INFO] processed {} total images".format(i))
       
        #split each row
        (label, imagePath) = row.strip().split(",")[-2:]
        
        #load each path
        imagePath = os.path.sep.join([basePath, imagePath])
        image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
        
        # resize the image to be 32x32 pixels and 
        # applying Contrast Limited Adaptive Histogram Equalization (CLAHE)
        image = transform.resize(image, (32, 32))
        image = exposure.equalize_adapthist(image, clip_limit=0.1)
        data.append(image)
        labels.append(int(label))
        
    # convering to np array
    data = np.array(data)
    labels = np.array(labels)

    return (data, labels)

In [5]:
# derive training and testing CSV paths
trainPath = os.path.sep.join([base_path, "Train.csv"])
testPath = os.path.sep.join([base_path, "Test.csv"])

# load the training and testing data
print("[INFO] loading training and testing data...")
(trainX, trainY) = load(base_path, trainPath)
(testX, testY) = load(base_path, testPath)

[INFO] loading training and testing data...
[INFO] processed 0 total images


D:\Soft\Anaconda3\lib\site-packages\skimage\util\dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


[INFO] processed 1000 total images
[INFO] processed 2000 total images
[INFO] processed 3000 total images
[INFO] processed 4000 total images
[INFO] processed 5000 total images
[INFO] processed 6000 total images
[INFO] processed 7000 total images
[INFO] processed 8000 total images
[INFO] processed 9000 total images
[INFO] processed 10000 total images
[INFO] processed 11000 total images
[INFO] processed 12000 total images
[INFO] processed 13000 total images
[INFO] processed 14000 total images
[INFO] processed 15000 total images
[INFO] processed 16000 total images
[INFO] processed 17000 total images
[INFO] processed 18000 total images
[INFO] processed 19000 total images
[INFO] processed 20000 total images
[INFO] processed 21000 total images
[INFO] processed 22000 total images
[INFO] processed 23000 total images
[INFO] processed 24000 total images
[INFO] processed 25000 total images
[INFO] processed 26000 total images
[INFO] processed 27000 total images
[INFO] processed 28000 total images
[

In [6]:
#setting parameters for training:
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 64

#this is an arbitary name file that was found with the dataset. 
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# one-hot encode the training and testing labels
numLabels = len(np.unique(trainY))
trainY = to_categorical(trainY, numLabels)
testY = to_categorical(testY, numLabels)

#accounting for the skewed classes
classTotals = trainY.sum(axis=0)
classWeight = dict()
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

In [8]:
temptrain = trainX
temptest = testX

In [18]:
trainX.shape

(39209, 32, 32)

In [9]:
trainY.shape

(39209, 43)

In [10]:
trX = trainX
teX = testX
# trY = trainY[:,:,:,np.newaxis]
# trY = testX[:,:,:,np.newaxis]

In [22]:
trX = np.expand_dims(trainX,3)
teX = np.expand_dims(testX,3)

In [24]:
trX.shape

(39209, 32, 32, 1)

In [25]:
#data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

# initialize and compiiling
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))
model = TrafficSignNet.build(width=32, height=32, depth=1,
        classes=numLabels)
model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

#early callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=5,
                                            restore_best_weights=True)

# train the network
print("[INFO] training network...")
H = model.fit(
    aug.flow(trX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=trX.shape[0] // BS,
    epochs=NUM_EPOCHS,
    class_weight=classWeight,
    callbacks=[callback],
    verbose=1)

[INFO] compiling model...
[INFO] training network...
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/30
610/612 [============================>.] - ETA: 0s - loss: 8.0373 - accuracy: 0.1605

ValueError: in user code:

    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1323 test_function  *
        return step_function(self, iterator)
    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\Soft\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1307 run_step  **
        outputs = model.test_step(data)
 

In [ ]:
X_train.shape

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
        predictions.argmax(axis=1), target_names=labelNames))

# save the network
timestr = time.strftime("%Y%m%d-%H%M%S")
print("[INFO] serializing network to '{}.{}'...".format(save_path,timestr))
# save_path = os.path.sep.join(save_path,)
model.save(os.path.sep.join([save_path,timestr]))

In [ ]:
# plot the training loss and accuracy
N = np.arange(0, callback.stopped_epoch+1)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)

# New Model

In [ ]:
# !pip install pandas

In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os
from sklearn.metrics import classification_report
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
base_path = "D:\GTSRB Dataset\gtsrb\\"
save_path = "./trafficsignnet.model"
plot_path = "./plot.png"
test_path = "./Test/"
example_path = "./examples"

In [ ]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 32
width = 32
channels = 1
classes = 43
n_inputs = height * width*channels
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 64

for i in range(classes) :
    path = base_path+"/train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a, cv2.IMREAD_GRAYSCALE)
#             print(image)
#             image_from_array = Image.fromarray(image, 'L')
            image = cv2.resize(image, (height, width))
            data.append(np.array(image))
#             print(image)
            labels.append(i)
        except AttributeError:
            print("Error")
#     print(data)
Cells=np.array(data)
labels=np.array(labels)

#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [ ]:
#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

#Using one hote encoding for the train and validation labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [ ]:
#accounting for the skewed classes
classTotals = y_train.sum(axis=0)
classWeight = dict()
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

In [ ]:
X_train.shape

In [ ]:
X_train = X_train[:,:,:,np.newaxis]
# teX = testX[:,:,:,np.newaxis]

In [ ]:
X_train.shape

In [ ]:
#Definition of the DNN model

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# )

In [ ]:
#data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

# initialize and compiiling
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))
# model = TrafficSignNet.build(width=32, height=32, depth=1,
#         classes=numLabels)
model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

#early callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=5,
                                            restore_best_weights=True)

# train the network
print("[INFO] training network...")
history = model.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    validation_data=(X_val, y_val),
    steps_per_epoch=X_train.shape[0] // BS,
    epochs=NUM_EPOCHS,
    class_weight=classWeight,
    callbacks=[callback],
    verbose=1)

In [ ]:
# #using ten epochs for the training and saving the accuracy for each epoch
# epochs = 20
# history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
# validation_data=(X_val, y_val))

In [ ]:
#Display of the accuracy and the loss values
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
#Predicting with the test data
y_test=pd.read_csv(base_path+"Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values


data=[]

for f in labels:
    image=cv2.imread(base_path+f, cv2.IMREAD_GRAYSCALE)
#             image_from_array = Image.fromarray(image, 'L')
#     print(base_path+f)
    image = cv2.resize(image, (height, width))
    data.append(np.array(image))

testX=np.array(data)
testX = testX.astype('float32')/255 
pred = model.predict(testX)

In [ ]:
pred = np.argmax(pred,axis=1)

In [ ]:
#Accuracy with the test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

In [ ]:
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX)
print(classification_report(y_test,pred, target_names=labelNames))

# save the network
timestr = time.strftime("%Y%m%d-%H%M%S")
print("[INFO] serializing network to '{}.{}'...".format(save_path,timestr))
# save_path = os.path.sep.join(save_path,)
model.save(os.path.sep.join([save_path,timestr]))

# Tester to load YOLO model and run detection and classification

In [ ]:
# import the necessary packages
from tensorflow.keras.models import load_model
from skimage import transform
from skimage import exposure
from skimage import io
from imutils import paths
import numpy as np
import argparse
import imutils
import random
import cv2
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
#function to plot an image
def plot_img(image, label,size=[6.4,4.8]):
    plt.figure(figsize=size)
    plt.axis(False)
    plt.title(label)
    plt.imshow(image)
    return

In [ ]:
#function to predict label for each input box

def predict_label(model, roi, labelNames):
    try:
        image = cv2.resize(roi,(32, 32))
    except:
        return "0"
    image = image[np.newaxis,:,:,np.newaxis]
    image = np.array(image).astype("float32") / 255.0
    
    # make predictions
    preds = model.predict(image)
    print(preds)
    print(np.max(preds))
    if np.max(preds)>0.75:
        j = preds.argmax(axis=1)[0]
        label = labelNames[j]
    else:
        label = "others"
    
    return label

In [ ]:
#function to draw boxes and labels on the first input image

def detected_image(image_path, bbs, labels):
    image = io.imread(image_path)
    for box,label in zip(bbs,labels):
        cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[0] + box[2]), int(box[1] + box[3])), [172 , 10, 127], 2)
        cv2.putText(image, label, (int(box[0] + box[2]), int(box[1] + box[3])), cv2.FONT_HERSHEY_DUPLEX,0.75, (255, 255, 0), 2)
        
    return image

In [ ]:
#function to crop box
def crop_roi(image,box):
    h, w = image.shape[:2]
    x_center, y_center = (box[0] * w), (box[1] * h)
    box_width, box_height = (box[2] * w), (box[3] * h)
    x_min, y_min = (x_center - box_width/2), (y_center - box_height/2)
    roi = image[int(y_min):int(y_min+box_height), int(x_min):int(x_min+box_width)]
    return roi, [x_min, y_min,box_width, box_height]

In [ ]:
image_path = "D:\GTSRB Dataset\gtsrb\captured_frames\\frame1214.jpg"
output_path = "D:\\GTSRB Dataset\\gtsrb\\test_output"
model_name = "./trafficsignnet.model\\20220116-111747"

In [ ]:
class_ids = []
confidences = []
boxes = []
rois = []
labels = []
bbs=[]

print("[INFO] loading model...")
model = load_model(model_name)

#yolo setup
net = cv2.dnn.readNet("yolov4-tiny_training_last_alz.weights", "yolov4-tiny_training.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# print(output_layers)
confidence_threshold = 0.5

#forward pass yolo
image = cv2.imread(image_path)
plot_img(image, "final", [16,12])
blob = cv2.dnn.blobFromImage(image, 0.004, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

#reading again since classfier performing better on skimage
image=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)

for out in outs:
    #     print(out.shape)
    for detection in out:
    #         print(len(detection))
        confidence = np.max(detection[5:])
        
        if confidence > confidence_threshold:
        #             print(confidence)
            roi, box = crop_roi(image,detection)
            confidences.append(float(confidence))
            rois.append(roi)
            boxes.append(box)



#adjust overlaps
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# load the label names
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

for i,roi in enumerate(rois):
    if i in indexes:
        label = predict_label(model, roi,labelNames)
        plot_img(roi, label,[3,3])
        labels.append(label)
        bbs.append(boxes[i])
        
image = detected_image(image_path, bbs, labels)    
plot_img(image, "final", [16,12])
io.imsave(os.path.join(output_path,image_path.split("\\")[-1]),image)

# Video Tester

In [ ]:
video_path = "D:\\GTSRB Dataset\\gtsrb\\video_input\\munich.mp4"
output_path = "D:\\GTSRB Dataset\\gtsrb\\video_output"
model_name = "./trafficsignnet.model\\20220116-090355"
path = os.path.join(output_path,video_path.split("\\")[-1])
os.makedirs(path, exist_ok=True)

In [ ]:
import time
start_time = time.time()
frame_count = 0
tfps = 30
cap = cv2.VideoCapture(video_path)
curr_frame = 0

print ("Set 1: ", str(round(time.time()-start_time, 2)))
start_time = time.time()

In [ ]:
print("[INFO] loading model...")
model = load_model(model_name)
print ("Set 2: ", str(round(time.time()-start_time, 2)))
start_time = time.time()

#yolo setup
net = cv2.dnn.readNet("yolov4-tiny_training_last_alz.weights", "yolov4-tiny_training.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# print(output_layers)
confidence_threshold = 0.5
print ("Set 3: ", str(round(time.time()-start_time, 2)))

In [ ]:
while(True):
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break
    elif np.count_nonzero(np.array(frame.shape))<3:
        continue
    i+=1
    # print ("Set 4: ", str(round(time.time()-start_time, 2)))
    if i%5 == 0:
        try:
            start_time = time.time()

            curr_frame +=1
            image_path = 'captured_frames/frame'+str(curr_frame)+'.jpg'
            cv2.imwrite('captured_frames/frame'+str(curr_frame)+'.jpg',frame)
            #         img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # print ("Set 3: ", str(round(time.time(), 2)))
            class_ids = []
            confidences = []
            boxes = []
            rois = []
            labels = []
            bbs=[]


            #forward pass yolo
            image = cv2.imread(image_path)
            blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            #reading again since classfier performing better on skimage
            image=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)

            for out in outs:
                for detection in out:
                    confidence = np.max(detection[5:])

                    if confidence > confidence_threshold:
                        roi, box = crop_roi(image,detection)
                        confidences.append(float(confidence))
                        rois.append(roi)
                        boxes.append(box)

            #adjust overlaps
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            for i,roi in enumerate(rois):
                if i in indexes:
                    label = predict_label(model, roi)
                    labels.append(label)
                    bbs.append(boxes[i])

            image = detected_image(image_path, bbs, labels)    
            io.imsave(path+"\\frame"+str(curr_frame),image)
            elapsed_time = time.time() - start_time
            fps = 1/elapsed_time
            print ("frame", curr_frame)
            print ("fps: ", str(round(fps, 2)))
        except ValueError:
            continue
cap.release()
cv2.destroyAllWindows()

In [ ]:
video_path = "D:\\GTSRB Dataset\\gtsrb\\video_input\\munich.mp4"
output_path = "D:\\GTSRB Dataset\\gtsrb\\video_output"
model_name = "./trafficsignnet.model"

In [ ]:
path = os.path.join(output_path,video_path.split("\\")[-1])
os.makedirs(path, exist_ok=True)
print(path+"\\frame"+str(curr_frame))
io.imsave(path+"\\frame"+str(curr_frame),image)